In [304]:
import os
import pandas as pd
import yfinance as yf
from ta.momentum import RSIIndicator, stochrsi_k
from ta.trend import MACD as md
from ta.volatility import AverageTrueRange as atr

# Technical Scanner

In [276]:
with open('ticker.csv') as f:
    lines = f.read().splitlines()
    for symbol in lines:
        print(symbol)
        data = yf.download(symbol, start="2020-01-01", end="2020-11-26")
        df = pd.DataFrame(data)

# for filename in os.listdir('datasets'):
#     symbol = filename.split(".")[0]
#     df = pd.read_csv('datasets/{}'.format(filename))
        if df.empty:
            continue

        RSI = RSIIndicator(close=df['Close'])
        df["RSI"] = RSI.rsi()

        Stochk = stochrsi_k(close=df['Close'])
        df["Stochk"] = Stochk

        MACD = md(close=df['Close'])
        df["MACD_diff"] = MACD.macd_diff()

        df["Signal"] = 0

BIG
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
DHI
[*********************100%***********************]  1 of 1 completed
VEEV
[*********************100%***********************]  1 of 1 completed
PFSI
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
LOW
[*********************100%***********************]  1 of 1 completed
WST
[*********************100%***********************]  1 of 1 completed
PLMR
[*********************100%***********************]  1 of 1 completed
ENTG
[*********************100%***********************]  1 of 1 completed
STMP
[*********************100%***********************]  1 of 1 completed
AMZN
[*********************100%***********************]  1 of 1 completed
PYPL
[*********************100%************

In [280]:
df

,Open,High,Low,Close,Adj Close,Volume,RSI,Stochk,MACD_diff,Signal
Date,,,,,,,,,,
2019-12-31,28.709999,29.080000,28.520000,28.719999,27.530489,672600,NaN,NaN,NaN,0
2020-01-02,28.959999,28.959999,27.549999,27.680000,26.533566,1418300,NaN,NaN,NaN,0
2020-01-03,27.430000,27.850000,27.129999,27.299999,26.169302,1028200,NaN,NaN,NaN,0
2020-01-06,27.049999,28.799999,26.809999,28.740000,27.549664,1597200,NaN,NaN,NaN,0
2020-01-07,28.410000,29.219999,28.350000,29.200001,27.990610,1209100,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
2020-11-19,48.500000,50.040001,48.360001,49.740002,49.740002,634700,52.561805,0.730768,0.016351,0
2020-11-20,49.900002,50.570000,49.090000,49.480000,49.480000,858300,51.712376,0.771436,0.075260,0
2020-11-23,49.810001,52.209999,49.590000,51.110001,51.110001,890200,56.462608,0.890500,0.214246,0


In [325]:
dataframes = {}
buys = {}
sells = {}
buy_list = []
buy_list_sl = []
buy_list_tp = []
sell_list = []
sell_list_tp = []
sell_list_sl = []

with open('sp500.csv') as f:
    lines = f.read().splitlines()
    for symbol in lines:
        print(symbol)
        data = yf.download(symbol, start="2020-01-01", end="2020-11-26")
        df = pd.DataFrame(data)

        if df.empty:
            continue

        RSI = RSIIndicator(close=df['Close'])
        df["RSI"] = RSI.rsi()

        Stochk = stochrsi_k(close=df['Close'])
        df["Stochk"] = Stochk

        MACD = md(close=df['Close'])
        df["MACD_diff"] = MACD.macd_diff()

        AvgTru = atr(high=df['High'],low=df['Low'],close=df['Close'],n=7)
        df["ATR"] = AvgTru.average_true_range()

        df["Signal"] = ""

        for i in range(0,len(df)):
            if df["RSI"].iloc[i] > 50 and df["Stochk"][i] > 50 and df["MACD_diff"][i] > 0:
                df["Signal"][i] = "Buy"
            elif df["RSI"].iloc[i] < 50 and df["Stochk"][i] < 50 and df["MACD_diff"][i] < 0:
                df["Signal"][i] = "Sell"
            else:
                df["Signal"][i] = "Neutral"

        if df["Signal"].iloc[-1] == "Buy" and (df["Signal"].iloc[-2] != "Buy"):
            buy_list.append(symbol)
            buy_list_sl.append(df["Close"].iloc[-1]-(df["ATR"].iloc[-1]*1.4))
            buy_list_tp.append(df["Close"].iloc[-1]+(df["ATR"].iloc[-1]*2.8))
        elif df["Signal"].iloc[-1] == "Sell" and (df["Signal"].iloc[-2] != "Sell"):
            sell_list.append(symbol)
            sell_list_sl.append(df["Close"].iloc[-1]+(df["ATR"].iloc[-1]*1.4))
            sell_list_tp.append(df["Close"].iloc[-1]-(df["ATR"].iloc[-1]*2.8))
        else:
            continue

        buys = pd.DataFrame({
            "Symbol" : buy_list,
            "SL" : buy_list_sl,
            "TP" : buy_list_tp
        })

        sells = pd.DataFrame({
            "Symbol" : sell_list,
            "SL" : sell_list_sl,
            "TP" : sell_list_tp
        })

***************100%***********************]  1 of 1 completed
HFC
[*********************100%***********************]  1 of 1 completed
HOLX
[*********************100%***********************]  1 of 1 completed
HD
[*********************100%***********************]  1 of 1 completed
HON
[*********************100%***********************]  1 of 1 completed
HRL
[*********************100%***********************]  1 of 1 completed
HST
[*********************100%***********************]  1 of 1 completed
HPQ
[*********************100%***********************]  1 of 1 completed
HUM
[*********************100%***********************]  1 of 1 completed
HBAN
[*********************100%***********************]  1 of 1 completed
HII
[*********************100%***********************]  1 of 1 completed
IEX
[*********************100%***********************]  1 of 1 completed
IDXX
[*********************100%***********************]  1 of 1 completed
INFO
[*********************100%***********************]  1 o

In [327]:
sells

,Symbol,SL,TP
0,CDW,137.743284,120.333439
1,MAS,56.176354,50.127291
2,PCAR,92.143229,82.173542
3,WRB,67.876514,59.126968


# Options Pricing Calculator

In [81]:
import mibian as mb
import requests
from bs4 import BeautifulSoup

TREASURY_URL = "https://www.marketwatch.com/investing/bond/tmubmusd03m?countrycode=bx"
OVERNIGHT_RATE = 0

r = requests.get(TREASURY_URL)
soup = BeautifulSoup(r.text, 'html.parser')

rfr = float(soup.find('bg-quote',{"class": "value"}).string)

In [181]:
TICKER = "SPY"
DATE = "2020-11-27"
STRIKE = 330.0

STK = yf.Ticker(TICKER)
opt = STK.option_chain(DATE)
optcc = opt.calls
impv = float(optcc[optcc["strike"]==STRIKE]["impliedVolatility"])

0.7485376708984376

In [93]:
SPY.info['previousClose']

363.22

In [182]:
sim_list = []
underlying_price = []

for i in range(364,390,2):
    underlying_price.append(i)
    sim = mb.BS([i,330,rfr,2],volatility=float(optcc[optcc["strike"]==i]["impliedVolatility"]))
    returns = (sim.callPrice - 32.45)*100
    sim_list.append(returns)
    track = pd.DataFrame(
            {"Underlying" : underlying_price,
            "Returns" : sim_list
            }
    )

In [147]:
import plotly.express as px

In [183]:
px.line(track, x="Underlying", y="Returns")